In [131]:
import requests
import json
from IPython.core.display import display, HTML

Set the drug that we want to find associations for

In [132]:
#myDrug = "INFLIXIMAB"
myDrug = "GEFITINIB"

Make the API call to find all the drugs

In [133]:
esURL = 'http://172.17.7.25:80/16.08_evidence-data-generic/_search'

query = json.dumps({
        "fields": ["disease.efo_info.label", "drug.molecule_name", "drug.id", "target.gene_info.symbol"],
        "query": {
            "match": {
                "drug.molecule_name": myDrug
            }
        }
    })

response = requests.get(esURL, data=query)
output = response.json()

#print out the json for debug
#print json.dumps(output, indent=2)

Collect information about diseases and targets associated with our drug

In [134]:
diseaseDict = {}
diseaseArray = []
targetDict = {}
targetArray = []

for data in output['hits']['hits']:
    
    #print data['evidence']['drug']['max_phase_for_all_diseases']['numeric_index']
    #print json.dumps(data, indent=2)

    for x in data['fields']['disease.efo_info.label']:
        diseaseDict[x] = 'true'
    for y in data['fields']['target.gene_info.symbol']:
        targetDict[y] = 'true'

#print "DRUG: " + myDrug

#print "\nDISEASES: "

for key in diseaseDict.keys():
    diseaseArray.append({'label': key})
#    print "\t" + key
    
#print "\nTARGETS:"

for key in targetDict.keys():
    targetArray.append({'label': key})
#    print "\t" + key
    #display(HTML('<h1>' + key + '</h1>'))


In [135]:
dataJSON = [{'label': "Diseases", 'weight': 1.0},
           {'label': "Targets", 'weight': 1.0}
        ]

dataJSON[0]["groups"] = diseaseArray
dataJSON[1]["groups"] = targetArray    


In [136]:
display(HTML("""
<!DOCTYPE html>
<html>
  <head>
    <title>FoamTree Quick Start</title>
    <meta charset="utf-8" />
  </head>

  <body>
  <center><h1>"""
  + myDrug +
   """</h1>
    <div id="visualization" style="width: 800px; height: 500px"></div></center>

    <script src="carrotsearch.foamtree.js"></script>
    <script>
      function init() {
        var foamtree = new CarrotSearchFoamTree({
          id: "visualization",
          dataObject: {
            groups:"""
             + 
             json.dumps(dataJSON)
             +
             """
          }
        });
      }
        
    init();
    </script>
  </body>
 """))
